# Analysis of the Professional Counter Strike Global Offensive Circuit
## CMSC 320 Final Project
## Richard Chen

# REQUIREMENTS

1500 words of prose, 150 lines of Python code, appropriate documentation, visualization, and links to any external information

## Motivation. 
Does the tutorial make the reader believe the topic is relevant or important (i) in general and (ii) with respect to data science?
## Understanding. 
After reading through the tutorial, does an uninformed reader feel informed about the topic? Would a reader who already knew about the topic feel like s/he learned more about it?
## Other resources. 
Does the tutorial link out to other resources (on the web, in books, etc) that would give a lagging reader additional help on specific topics, or an advanced reader the ability to dive more deeply into a specific application area or technique?
## Prose. 
Does the prose portion of the tutorial actually add to the content of the deliverable?
## Code. 
Is the code well written, well documented, reproducible, and does it help the reader understand the tutorial? Does it give good examples of specific techniques?
## Subjective evaluation. 
If somebody linked to this tutorial from, say, Hacker News, would people actually read through the entire thing?